<a href="https://colab.research.google.com/github/seonae0223/Deep_Learning/blob/main/04_PyTorch_Model_Training_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.FashionMNIST(
    root="data", # 데이터를 저장할 root 디렉토리
    train=True, # 훈련용 데이터 설정
    download=True, # 다운로드
    transform=ToTensor() # 이미지 변환. 여기서는 TorchTesnor로 변환시킵니다.
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader  = DataLoader(test_data, batch_size=64, shuffle=False)

In [ ]:
from torch import nn

class NeuralNetwork(nn.Module):

  def __init__(self):
    super(NeuralNetwork, self).__init__()

    self.flatten = nn.Flatten()

    # nn.Sequential을 이용해 연속되는 레이어의 구조를 구성
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 128),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(128, 10),
    )

  def forward(self, x):
    x = self.flatten(x)
    y = self.linear_relu_stack(x)

    return y

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)


# 모델 훈련

파이토치의 모델 훈련을 위해서는 손실함수(Loss/모델의 성능), 최적화 함수(Optimizer)를 등록해야 합니다. 특히 최적화 함수를 사용하기 위해서는 `model.parameters()` 메소드를 이용해 최적화 대상 파라미터(가중치, bias)를 지정해주면 됩니다.

In [ ]:
# ⭐아주 주요한 부분
loss_fn = nn.CrossEntropyLoss()

# optimizer : 경사하강법을 수행하기 위한 함수. 경사하강법은 어디에 수행? W(가중치), b(bias) -> parameters
# model에서 파라미터를 꺼내서 최적화 함수에 등록
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

In [ ]:
# 훈련 과정(훈련 루프 정의)
# 1. dataloader에서 데이터를 꺼낸다.
# 2. 데이터를 모델에 통과시킨다. (순전파를 통한 추론 - prediction(inference))
# 3. 얻어낸 예측값을 이용해서 loss를 계산
# 4. 역전파를 통한 미분값을 계산
# 5. 얻어낸 미분 값으로 경사하강법을 수행(최적화)

def train_loop(dataloader, model, loss_fn, optimizer):
  # 데이터 로더에 있는 데이터 세트의 길이 가져오기

  size = len(dataloader.dataset)

  # 중요! model을 모드로 설정.⭐️
  model.train()
  # 데이터 꺼내기
  for batch, (X, y) in enumerate(dataloader):
    # 현재 데이터 로더에 있는 데이터는 cpu에 존재하고 있기 때문에 이 데이터들을 GPU로 옮긴다.
    #   모델이 위치한 곳과 데이터가 위치한 곳을 동일하게 맞춰준다.
    X, y = X.to(device), y.to(device)

    # 순전파 수행
    pred = model(X)

    # 손실 계산
    loss = loss_fn(pred, y) # 자동으로 소프트맥스가 적용됨.

    # 역전파 수행(미분값 없어내기)
    optimizer.zero_grad() # 기존에 남아있던 기울기를 제거(이전 배치의 기울기가 남아있으면 정확한 기울기를 구해내기가 힘듦)
    loss.backward() # 역전파. loss가 Leaf
    optimizer.step() # 구한 미분값을 토대로 최적화를 수행(경사하강법)

    # 배치가 100번 돌 때마다 화면에 출력
    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"Train Loss : {loss:>7f} [ {current:>5d} / {size:>5d} ]")

In [ ]:
# 추론을 위한 테스트 과정(테스트 루프) 정의
# 1. 테스트 데이터 로더에서 데이터 꺼내기
# 2. 데이터를 모델에 통과(순전파)시켜서 예측값 얻어내기
# 3. 성능(metric) 계산. - Loss, Accuracy 계산
#   - 배치 별 평균 성능 계산
def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0

  # 중요! 평가모드(추론모드) 설정
  model.eval()

  # 추론 과정은 기울기를 구할 필요가 없어요
  with torch.no_grad():
    for X, y in dataloader:

      # 모델과 데이터는 항상 같은 환경에서 사용되어야 한다.
      X, y = X.to(device), y.to(device)

      # 예측
      pred = model(X)

      # Loss 계산
      test_loss += loss_fn(pred, y).item()

      # 맞춘거 개수 합치기
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  # 배치 개수 구하기
  num_batches = len(dataloader)

  # 배치 별 loss, accuracy의 평균 구하기
  test_loss /= num_batches
  correct /= size

  print(f"Test Error : \n Accuracy : {(100*correct):>0.1f}%, Avg Loss : {test_loss:>8f}\n")


# 훈련 수행

In [ ]:
epochs = 10

for i in range(epochs):
  print(f"Epochs {i + 1}\n......................")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)

print("Done!!!")

Epochs 1
......................
Train Loss : 2.195719 [     0 / 60000 ]
Train Loss : 2.213618 [  6400 / 60000 ]
Train Loss : 2.184967 [ 12800 / 60000 ]
Train Loss : 2.193613 [ 19200 / 60000 ]
Train Loss : 2.199875 [ 25600 / 60000 ]
Train Loss : 2.187859 [ 32000 / 60000 ]
Train Loss : 2.189427 [ 38400 / 60000 ]
Train Loss : 2.188310 [ 44800 / 60000 ]
Train Loss : 2.186734 [ 51200 / 60000 ]
Train Loss : 2.163091 [ 57600 / 60000 ]
Test Error : 
 Accuracy : 28.2%, Avg Loss : 2.165968

Epochs 2
......................
Train Loss : 2.166116 [     0 / 60000 ]
Train Loss : 2.144289 [  6400 / 60000 ]
Train Loss : 2.158753 [ 12800 / 60000 ]
Train Loss : 2.120757 [ 19200 / 60000 ]
Train Loss : 2.136964 [ 25600 / 60000 ]
Train Loss : 2.152539 [ 32000 / 60000 ]
Train Loss : 2.117127 [ 38400 / 60000 ]
Train Loss : 2.159462 [ 44800 / 60000 ]
Train Loss : 2.118912 [ 51200 / 60000 ]
Train Loss : 2.148128 [ 57600 / 60000 ]
Test Error : 
 Accuracy : 36.9%, Avg Loss : 2.123652

Epochs 3
...................